-----------------------
## 1) New way of scraping  11.4.2024
-----------------------

In [1]:
import os
os.chdir("c:\\Users\\jirka\\Documents\\MyProjects\\Sreality")

from run import Runner
runner = Runner()

## Complete scraping + updating DB

In [ ]:
runner.run_complete_scraping(scrape_prodej_byty=True, scrape_all=True)

In [2]:
#testik = runner.scrape_prices_and_details(scrape_prodej_byty=False,
#                                            scrape_all=True)

There are max 95 pages with 999 results each
SCRAPING Prices


100%|██████████| 95/95 [04:29<00:00,  2.84s/it]


SCRAPING Estate detils.


100%|██████████| 12/12 [00:12<00:00,  1.01s/it]


In [ ]:
#runner.input_all_prices_to_db()

## Test diagnostics

In [ ]:
from diagnostics.diagnostics import Diagnostics

diag = Diagnostics()

result = diag.summary_new_estates()


In [ ]:
display(result)

In [ ]:
import json 

runner.mailing.send_email(subject=f'SCRAPING SREALITY SUMMARY for {result["latest_timestamp"]}',
                        message_text=json.dumps(result))


## Testing requests

In [ ]:
# Just testing count
headers = {"User-Agent": "Mozilla/5.0"}

with requests.Session() as session:
    url = "https://www.sreality.cz/api/cs/v2/estates/count"
    result =  session.request(method="GET", url=url, headers=headers).json()
    print(result)
    url_filter = f"https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&per_page=1&page=1"
    result2 =  session.request(method="GET", url=url_filter).json()
    print(result2)

In [ ]:
headers = {"User-Agent": "Mozilla/5.0"}

with requests.Session() as session:
    url = "https://www.sreality.cz/api/cs/v2/estates/95413580"
    result =  session.request(method="GET", url=url, headers=headers).json()
    display(result)


## Test Utils - timestamp and mail

In [ ]:
from run import Runner
runner = Runner()

In [ ]:
timestamp, _ = runner.utils.generate_timestamp()
print(timestamp)
#data_manager._insert_new_estate(df_
# new, timestamp)

In [ ]:
runner.mailing.send_email("hallo", "toto je mesidz")



In [ ]:
runner.mailing.send_email_with_attachment("hallo", "toto je mesidz", 'scraping.log')


## Test Prepare Geo-JSONs

In [ ]:
source_folder= "c:\\Users\\jirka\\Documents\\MyProjects\\Sreality\\data_2024\\estate_details"
files = os.listdir(source_folder)
file_info = []

for file_name in files:
        file_path = os.path.join(source_folder, file_name)
        if os.path.isfile(file_path):
            size = os.path.getsize(file_path)
            file_info.append((file_name, size))

# Sort files by size
sorted_files = sorted(file_info, key=lambda x: x[1])

sorted_names = []
for file_name, size in sorted_files:
    sorted_names.append(file_name)
    print(f"{file_name}: {size} bytes")

In [ ]:
runner.geodata.enrich_jsons_with_geodata(list_of_jsons=None)

## Test table managment

In [ ]:
runner = Runner()
runner.input_all_estates_to_db()

In [ ]:
runner = Runner()
runner.input_all_prices_to_db()

In [ ]:
d = runner.data_manager.get_all_rows("estate_detail")
print(len(d))
d_codes = d["code"].unique()
print(len(d_codes))
print(d_codes[0])
print(type(d_codes[0]))

display(d.head(2))
print(len(d.columns))
#display(sorted(d.columns))


In [ ]:
# price_history, estate_detail
runner.data_manager.clear_table("X")

In [ ]:
runner.data_manager.drop_table("x")

In [ ]:
runner.data_manager.create_table("x")

In [ ]:
upload = pd.DataFrame(data = [[42, "Prodej luxsního bejváku", "Byt 14+kk daleko od centra, ale v klidu.",
                               1, 6, 1, 111, "M&M reality", 2, 49.827491575, 18.1654443,
                               0, 0, 111083, 65, 14829, 12, 10, 4730, 112,
                               0, 0, 0, 0, 0, 2, 0, 79, "Družstevní", 0, 1, 1, 0,
                               3, 0, 870824, 1, 2,
                               ### Not mandatory:
                               "Cena je finální", 123,
                               "Dnes", "cihla", "Novostavba", "Družstevní", "2.patro",
                               42, 43, 2, "true"
                               ]],
                      columns=['code', 'description', 'meta_description', 
                               'category_main_cb', 'category_type_cb', 'category_sub_cb', 'broker_id', 'broker_company',
                            'furnished', 'latitude', 'longitude', 'locality', 'city', 'district',
                            'region', 'object_type', 'parking_lots', 'locality_street_id',
                            'locality_district_id', 'locality_ward_id', 'locality_region_id',
                            'locality_quarter_id', 'locality_municipality_id',
                            'locality_country_id', 'terrace', 'balcony', 'loggia', 'basin',
                            'cellar', 'building_type', 'object_kind', 'ownership', 'low_energy',
                            'easy_access', 'building_condition', 'garage', 'room_count_cb', 'energy_efficiency_rating_cb', 
                            ### Not mandatory:
                            'note_about_price', 'id_of_order',
                            'last_update', 'material', 'age_of_building', 'ownership_type', 'floor',
                            'usable_area', 'floor_area', 'energy_efficiency_rating', "no_barriers"
                            ])

runner.data_manager._insert_new_estate(upload, "2024-04-21 23:22:44")

#TODO: how about unit test for this?
#? should fail predictably as it breakes foreign key constraints i made:)
upload = pd.DataFrame(data = [["911", "4242", "12345000"]],
                      columns=["estate_id", "batch_id", "price"])

runner.data_manager.insert_new_price(upload)

## Extra info

In [ ]:
describe = d["id"].describe()
describe

In [ ]:
print(describe["min"])

print(d["id"].min())

## Test Scheduler

In [ ]:
import datetime as dt

from scheduler import Scheduler

score = 1

def foo():
    print(f"foo called at {dt.datetime.now()}")

schedule = Scheduler()

schedule.cyclic(dt.timedelta(seconds=6), foo)

In [ ]:
print(schedule)

In [ ]:
import time

while True:
    schedule.exec_jobs()
    time.sleep(1)

## Konec - Bonusy, jednorázovky

In [ ]:

# One-time - all JSONs - rename key "code" into "estate_id", so it is unified

import os
import json

#folder_path = 'C://Users//jirka//Documents//MyProjects//Sreality//data_2024//estate_details'

json_files = [f for f in os.listdir(folder_path)]

for json_file in json_files:
    file_path = os.path.join(folder_path, json_file)
    
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    for item in data:
        if 'code' in item:
            item['estate_id'] = item.pop('code')
    
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

In [ ]:
# One-time - all CSVs - rename column "code" into "estate_id", so it is unified

import os
import pandas as pd

#folder_path = 'C://Users//jirka//Documents//MyProjects//Sreality//data_2024//prices'

csv_files = [f for f in os.listdir(folder_path)]
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    df = pd.read_csv(file_path, sep=";")
    
    df.rename(columns={'code':'estate_id'}, inplace=True)
    
    df.to_csv(file_path, sep=";", encoding="utf-8", index=False)
        



In [ ]:
# One-time scrape all locality info for all old estate_details if still available

import os
os.chdir("c:\\Users\\jirka\\Documents\\MyProjects\\Sreality")

from run import Runner
runner = Runner()

timestamp, to_save = runner.utils.generate_timestamp()
print(timestamp, to_save)

d = runner.data_manager.get_all_rows("estate_detail")
df_missing = d["estate_id"].unique()
#TODO: musím si někam do složky oddělit ty, které teď ve 22:54 tvořím. 
#todo: a příště si je načíst a zjistit co už mám a odečíst to od tady těch všech v DB

runner.scraper.scrape_specific_estates(df_missing, timestamp)